In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# 使用標註者完全一致的數據集
dataset = load_dataset("takala/financial_phrasebank", "sentences_allagree", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

加載 BERT

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

# 加載 BERT 分詞器和模型
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 三個情緒類別

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


資料預處理

In [4]:
def preprocess_data(examples):
    # 使用分詞器將文本轉換為 BERT 模型可接受的格式
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# 應用預處理函數到數據集
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [5]:
# 定義預處理函數
def preprocess_data(examples):
    # 使用分詞器將文本轉換為 BERT 模型可接受的格式
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# 應用預處理函數到數據集
encoded_dataset = dataset.map(preprocess_data, batched=True)

# 分割資料集
split_dataset = encoded_dataset["train"].train_test_split(test_size=0.2)

# 獲取分割後的訓練集和測試集
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# 設定格式
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

定義參數

In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 定義 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


模型訓練

In [7]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,No log,0.164920
2,No log,0.180854
3,0.230600,0.184936


TrainOutput(global_step=681, training_loss=0.18089075270553343, metrics={'train_runtime': 162.3863, 'train_samples_per_second': 33.457, 'train_steps_per_second': 4.194, 'total_flos': 357373799629056.0, 'train_loss': 0.18089075270553343, 'epoch': 3.0})

模型評估

In [9]:
trainer.evaluate()

{'eval_loss': 0.18493562936782837,
 'eval_runtime': 3.3417,
 'eval_samples_per_second': 135.56,
 'eval_steps_per_second': 17.057,
 'epoch': 3.0}

In [13]:
import torch
# 確保模型位於正確的設備上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict(text):
    # 將輸入數據移動到與模型相同的設備
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 移動到模型的設備
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(dim=-1)
    labels = ["negative", "neutral", "positive"]
    return labels[predictions.item()]

# 測試模型的預測
print(predict("The company's profits have increased significantly this quarter.")) # 這家公司本季度的利潤大幅增長。
print(predict("The stock has been performing poorly for the last few months.")) # 這支股票在過去幾個月的表現一直很差。
print(predict("The company reported record-breaking profits this quarter, exceeding all expectations.")) # 	這家公司本季度報告了破紀錄的利潤，超出了所有預期。
print(predict("The stock plummeted after the company failed to meet its revenue targets.")) # 這家公司未能達到收入目標後，其股價暴跌。
print(predict("The company announced its plans to release its quarterly earnings next week.")) # 這家公司宣布計劃在下週發布其季度收益報告。

positive
negative
positive
negative
neutral
